# RDD 최적화 예제

이 노트북은 Apache Spark RDD의 최적화 기법들을 설명합니다.

## 환경 설정

In [ ]:
#!sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

In [15]:
import findspark
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## 1. Persist/Cache 활용

여러 번 사용되는 RDD는 메모리에 캐시하여 재계산을 방지합니다.



```
사용법:
rdd_cached = rdd.cache()

사용 예시:
반복적인 알고리즘에서 동일한 RDD를 여러 번 사용할 때
대화형 분석에서 동일한 데이터셋을 반복 조회할 때
복잡한 변환 연산 후 결과를 여러 번 사용할 때
```


공식 도큐먼트: \
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.cache.html \
https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence


In [16]:
import time

# 데이터셋 설정
numbers = sc.parallelize(range(1, 100000))
# TODO: 위 설명을 참고하여 numbers를 캐시하세요
numbers_cached = sc.parallelize(range(1, 100000)).cache()

# 캐시되지 않은 RDD로 여러 번 반복
print("Without caching:")
for i in range(3):
    start_time = time.time()
    sum_val = numbers.sum()
    mean_val = numbers.mean()
    max_val = numbers.max()
    end_time = time.time()
    print(f"Iteration {i+1} time:", end_time - start_time, "sec")

print("\nWith caching:")
# 첫 번째 실행에서 캐싱
_ = numbers_cached.count()  # 캐싱 강제 실행

# 캐시된 RDD로 여러 번 반복
for i in range(3):
    start_time = time.time()
    sum_val = numbers_cached.sum()
    mean_val = numbers_cached.mean()
    max_val = numbers_cached.max()
    end_time = time.time()
    print(f"Iteration {i+1} time:", end_time - start_time, "sec")

Without caching:
Iteration 1 time: 1.2308361530303955 sec
Iteration 2 time: 1.212766408920288 sec
Iteration 3 time: 1.3975341320037842 sec

With caching:
Iteration 1 time: 1.6457555294036865 sec
Iteration 2 time: 1.5648951530456543 sec
Iteration 3 time: 2.511554718017578 sec


이 결과를 분석해보면 흥미로운 점이 있습니다:



```
캐시되지 않은 RDD:

첫 번째 반복: 36.54초 (매우 긴 시간)
두 번째 반복: 1.20초
세 번째 반복: 1.34초


캐시된 RDD:

첫 번째 반복: 1.42초
두 번째 반복: 1.75초
세 번째 반복: 1.54초
```


여기서 볼 수 있는 특이한 점은:

캐시되지 않은 RDD의 첫 번째 실행이 매우 느린 반면, 이후 실행은 오히려 캐시된 버전과 비슷하거나 더 빠릅니다. 이는 Spark나 시스템 레벨에서 자체적인 최적화가 일어나고 있음을 의미합니다.

조금 더 정확한 비교를 위해 복잡한 연산을 추가해보겠습니다.

In [18]:
import time

# 더 복잡한 연산을 수행하는 함수
def complex_operation(x):
    # 의도적으로 복잡한 연산 추가
    result = x
    for _ in range(10):
        result = (result * 2) % 100000
    return result

# 복잡한 연산 사용
numbers = sc.parallelize(range(1, 100000))
numbers_transformed = numbers.map(complex_operation)
# TODO: 위 설명을 참고하여 numbers를 캐시하세요
numbers_cached = numbers.map(complex_operation).cache()

print("Without caching:")
for i in range(3):
    start_time = time.time()
    # 여러 연산 수행
    sum_val = numbers_transformed.sum()
    count_val = numbers_transformed.count()
    max_val = numbers_transformed.max()
    end_time = time.time()
    print(f"Iteration {i+1} time: {end_time - start_time:.2f} sec")

print("\nWith caching:")
# 첫 번째 실행에서 캐싱
_ = numbers_cached.count()

for i in range(3):
    start_time = time.time()
    # 여러 연산 수행
    sum_val = numbers_cached.sum()
    count_val = numbers_cached.count()
    max_val = numbers_cached.max()
    end_time = time.time()
    print(f"Iteration {i+1} time: {end_time - start_time:.2f} sec")

Without caching:
Iteration 1 time: 2.29 sec
Iteration 2 time: 1.68 sec
Iteration 3 time: 1.17 sec

With caching:
Iteration 1 time: 0.78 sec
Iteration 2 time: 0.79 sec
Iteration 3 time: 0.84 sec


성능 차이가 발생하는 이유:

* 연산 재사용:  
 * 캐시되지 않은 경우: 매번 complex_operation을 다시 실행
 * 캐시된 경우: 변환된 결과를 메모리에서 직접 읽음


* 데이터 지역성:
 * 캐시된 데이터는 메모리에 있어 접근이 빠름
 * 캐시되지 않은 경우는 매번 데이터를 처리하고 변환해야 함


* 복잡한 연산의 영향:
 * complex_operation이 각 요소에 대해 10번의 반복 연산을 수행
 * 이로 인해 캐싱의 이점이 더 명확하게 드러남